Required Packages

In [2]:
# numpy==1.22.3
# pandas==1.4.2
# python-dateutil==2.8.2
# pytz==2022.1
# pywin32==303
# pyxlsb==1.0.9
# six==1.16.0
# xlwings==0.27.5

In [3]:
#import required modules
import os
import re
import asyncio
import functools
import pandas as pd

from cluster.cluster import Cluster
from concurrent.futures import ThreadPoolExecutor
from dataframes.dataframe_optimized import DataFrameOptimized
from utils import constants as const, index as utils

Functions

In [4]:
async def get_bases(sources: dict[str, str]) -> 'tuple(list[str], list[DataFrameOptimized])':
    """Get DataFrames of sources

    Args:
        sources (dict[str, str]): dict of sources 

    Returns:
        list[DataFrameOptimized]: [description]
    """
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(max_workers=4) as executor:

        futures = []
        keys = []

        for key, source in sources.items():
            path = results[key].split("|") #list[base, ...] - key is a name of base
            if len(path) == 1:
                path = path[0]
            keys.append(key)
            futures.append(loop.run_in_executor(executor, functools.partial(Cluster.preprocess_base, **{"path": path, "properties": source})))

        results = await asyncio.gather(*futures)
    
    return keys, results

def get_predeterminated_files(_path: str):
    
        found = {
            "base_socios": "",
            "base_coordenadas": "",
            "base_universo_directa": "",
            "base_universo_indirecta": "",
            "base_consulta_directa": "",
            "base_consulta_indirecta": ""
        }
        for (dirpath, dirnames, filenames) in os.walk(_path):
            for file in filenames:
                if re.search("socio", file, re.IGNORECASE):
                    found["base_socios"] = os.path.join(_path, file)
                elif re.search("coord", file, re.IGNORECASE):
                    found["base_coordenadas"] = os.path.join(_path, file)
                elif re.search(r"universo\s+direc", file, re.IGNORECASE):
                    found["base_universo_directa"] = os.path.join(_path, file)
                elif re.search(r"universo\s+indirec", file, re.IGNORECASE):
                    found["base_universo_indirecta"] = os.path.join(_path, file)

            for dir in dirnames:
                files = []
                root_dir = os.path.join(_path, dir)
                for _file in os.listdir(root_dir):
                    files.append(os.path.normpath(os.path.join(root_dir, _file)))
                if re.search("indirecta", dir, re.IGNORECASE):
                    found["base_consulta_indirecta"] = "|".join(files)
                elif re.search("directa", dir, re.IGNORECASE):
                    found["base_consulta_directa"] = "|".join(files)
        return found

In [5]:
#load configfile
config = utils.get_config(os.path.join(const.ROOT_DIR, "config.yml"))

principal process

In [ ]:
files_found = get_predeterminated_files(os.path.join(const.ROOT_DIR, "files/Bases"))
sources = config["sources"]

# bases = get_bases(sources)  

bases = {f"{file.split('.')[0]}": DataFrameOptimized(pd.read_csv(os.path.join(const.ROOT_DIR, f"files/temp/{file}"))) for file in os.listdir(os.path.join(const.ROOT_DIR, f"files/temp"))}
# bases = {f"{keys[idx]}": result for idx, result in enumerate(results)}

In [7]:
final_base = Cluster()
await final_base.merge_all(bases, config["order_base"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17145 entries, 0 to 17144
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cod_cliente  17145 non-null  int64 
 1   cod_loc      17145 non-null  int64 
 2   cod_agente   17145 non-null  object
 3   atencion     17145 non-null  object
 4   socios       17145 non-null  bool  
dtypes: bool(1), int64(2), object(2)
memory usage: 552.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263974 entries, 0 to 263973
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   cod_cliente  263974 non-null  object
 1   latitud      263974 non-null  object
 2   longitud     263974 non-null  object
 3   cod_agente   263974 non-null  object
 4   cod_ecom     263974 non-null  object
dtypes: object(5)
memory usage: 10.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17145 entries, 0 to 17144
Data columns (total 5 c

TypeError: 'filter' object is not subscriptable